In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=1eFtDy2OCElZDXZLRYuM0DnQO31NEAk9q4kpQUsj1bk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5InJQG-VpuzguKqLX7j_2X99MoF_x4PvBXmumf-hV4kb7TkbxWWgI

Successfully saved authorization token.


In [ ]:
import os
from google.colab import drive
drive.mount('/gdrive')
try:
    os.symlink('/gdrive/My Drive', '/content/gdrive')
except:
    pass
!ls -l /content/gdrive/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
total 26
drwx------ 2 root root 4096 Feb 24 11:59 EFNET
drwx------ 6 root root 4096 Jun  7 08:07 gee
drwx------ 2 root root 4096 Jan 31  2020 google_AI
drwx------ 2 root root 4096 Feb 21 14:16 HRNET
drwx------ 2 root root 4096 Mar 17 09:02 NET
drwx------ 2 root root 4096 Feb 23 14:00 RESNET
-rw------- 1 root root 1290 Jun 18 05:52 Untitled0.ipynb


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
import os 
import tensorflow as tf
try:
    os.chdir("/content/gdrive/gee")
except:
    pass
!ls

drive		 Efficiunet_r.py  GetGeeMap.ipynb  __pycache__
Efficiunet_b.py  gee		  MadeRecord.py    Untitled0.ipynb
Efficiunet_g.py  gee_model_3	  model1.h5	   Untitled1.ipynb
Efficiunet.py	 GEEUSED.ipynb	  model2.h5


In [ ]:
# constant
BANDS = ['B7', 'B3', 'B2']
RESPONSE = 'impervious'
FEATURES = BANDS + [RESPONSE]
list=[i for i in range(2015,2020)]
# month_end=["01-31","02-28","03-31","04-30","05-31","06-30","07-31","08-31","09-30","10-31","11-30","12-31"]
# month_start=["01-01","02-01","03-01","04-01","05-01","06-01","07-01","08-01","09-01","10-01","11-01","12-01"]
month_end=["03-31","06-30","09-30","12-31"]
month_start=["01-01","04-01","07-01","10-01"]
user_folder = 'users/qxcnwu'
bj_image_base = 'effunet_demo'
bj_kernel_buffer = [128, 128]
BUCKET="qxcgee"

# gee上取样点集合
k_list=[[98.08390445488186,28.434290895745573],[98.74308414238186,30.08277155520017],[93.97739579181439,31.911025577362487],[95.52646805743939,31.994920953432977],[97.35019852618939,31.859718481536852]]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 1024
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# model of effciunet
MODEL_DIR = 'gee_model_3/'
m = tf.keras.models.load_model(MODEL_DIR)

In [ ]:
def doExport(out_image_base, kernel_buffer,FOLDER):
  """Run the image export task.  Block until complete.
    .clip(ee.Geometry.Polygon([[94.60662921680765,30.401055969248482],[95.04608234180765,30.401055969248482],[95.04608234180765,30.651840122638728],[94.60662921680765,30.651840122638728],[94.60662921680765,30.401055969248482]]))
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = image.select(["B7","B3","B2"]),
    description = out_image_base,
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + out_image_base,
    scale = 30,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Cloud Storage...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

In [ ]:
def doPrediction(out_image_base, user_folder, kernel_buffer,FOLDER):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}

  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()

  from pprint import pprint

  import json
  # Load the contents of the mixer file to a JSON object.
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  import numpy as np
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches=0
  from tqdm import tqdm
  for predictionPatch in tqdm(predictions):
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]
    # Create an example.
    predictionPatchs=np.argmax(predictionPatch,-1)
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatchs.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

In [ ]:
flag=0
for k in range(12):
    flag+=1
    for i in list:
        for j in range(1,5):
            start_date=str(i)+'-'+month_start[j-1]
            end_date=str(i)+'-'+month_end[j-1]
            print(start_date,end_date,' starting!!...')
            image=ee.Image(ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate(start_date, end_date).filterBounds(ee.Geometry.Point(k[0],k[1])).sort('CLOUD_COVER').first())
            try:
                FLODER=str(i)+'_'+str(j)+"_"+str(flag)
                # 以下两部最好是分别使用 以寻求最佳的GPU使用策略
                bj_image_base='effunet_demo_left'+FLODER
                # doExport(bj_image_base, bj_kernel_buffer,FLODER)
                doPrediction(bj_image_base, user_folder, bj_kernel_buffer,FLODER)
            except:
                print(start_date,end_date)

2015-01-01 2015-03-31  starting!!...
2015_1_2 effunet_demo_left2015_1_2
Looking for TFRecord files...
Running predictions...
49/49 [==============================] - 68s 1s/step


100%|██████████| 49/49 [00:11<00:00,  4.34it/s]


Instructions for updating:
non-resource variables are not supported in the long term
Started upload task with ID: SVYT4SS4ZVYUDBNRGCNRDDQU
2015-04-01 2015-06-30  starting!!...
2015_2_2 effunet_demo_left2015_2_2
Looking for TFRecord files...
Running predictions...
49/49 [==============================] - 67s 1s/step


100%|██████████| 49/49 [00:11<00:00,  4.30it/s]


Instructions for updating:
non-resource variables are not supported in the long term
Started upload task with ID: 2YL6J5BG54JSY5Z4QQBHIAYM
2015-07-01 2015-09-30  starting!!...
2015_3_2 effunet_demo_left2015_3_2
Looking for TFRecord files...
Running predictions...
49/49 [==============================] - 66s 1s/step


...

Instructions for updating:
non-resource variables are not supported in the long term
Started upload task with ID: WSOVIACBCXKDMTEJ4B4SLZUF
2016-01-01 2016-03-31  starting!!...
2016_1_11 effunet_demo_left2016_1_11
Looking for TFRecord files...
Running predictions...
49/49 [==============================] - 59s 1s/step


100%|██████████| 49/49 [00:11<00:00,  4.35it/s]


Instructions for updating:
non-resource variables are not supported in the long term
Started upload task with ID: KAM3WD6E6VCHEK2Z3YPAEJIS
2016-04-01 2016-06-30  starting!!...
2016_2_11 effunet_demo_left2016_2_11
Looking for TFRecord files...
Running predictions...
49/49 [==============================] - 59s 1s/step


100%|██████████| 49/49 [00:11<00:00,  4.34it/s]


Instructions for updating:
non-resource variables are not supported in the long term
Started upload task with ID: T7RF27AFFHQK7WHSMHPLYXUM
2016-07-01 2016-09-30  starting!!...
2016_3_11 effunet_demo_left2016_3_11
Looking for TFRecord files...
Running predictions...
49/49 [==============================] - 56s 1s/step
